In [9]:
import numpy as np
import qgutils as qg
import sys

In [2]:
def get_psi(p, f0):
    
    '''
    parameters
    
    p : array[nz, nx, ny]
    f0 : scalar 
    
    '''
    
    nd = p.ndim
    
    if nd < 3:
        print("not handling less than 3D arrays")
        sys.exit(1)
        
    psi = qg.interp_on_c(p)/f0
    
    return psi


In [1]:
def reduce_z_dim(p, dh):
    
    nd = p.ndim
    
    if nd < 3:
        print("not handling less than 3D arrays")
        sys.exit(1)
    
    nz,ny,nx = p.shape
    p_use = np.ones((nz-1,ny,nx))
    
    p_use[0] = ( p[0] * dh[0,None,None]  +  p[1] * dh[1,None,None] )   /  ( dh[0,None,None] + dh[1,None,None] )
    p_use[1] = ( p[1] * dh[1,None,None]  +  p[2] * dh[2,None,None] )   /  ( dh[2,None,None] + dh[1,None,None] )
    
    return p_use

In [1]:
def div_x(p, Delta):
    
    p = qg.pad_bc(p, bc='dirichlet')
   
    dix = (p[...,1:-1,:-2] - p[...,1:-1,2:])/(2*Delta)
    
    return dix

In [2]:
def div_y(p, Delta):
    
    p = qg.pad_bc(p, bc='dirichlet')
   
    diy = (p[...,2:,1:-1] - p[...,:-2,1:-1])/(2*Delta)
    
    return diy

In [7]:
def div_z(psi,dh,f0):

    dhi = 0.5*(dh[1:] + dh[:-1])

    nd = psi.ndim
    nz,ny,nx = psi.shape

    if nd == 1:
        b = -np.diff(psi,1,0)/dhi[:]
    elif nd == 2: 
        b = -np.diff(psi,1,0)/dhi[:,None]
    elif nd == 3: 
        if nz==4 :
            b = -np.diff(psi,1,0)/dh[:,None,None]
        else :
            b = -np.diff(psi,1,0)/dhi[:,None,None]
    
        

    return b.squeeze()



In [10]:
def integral_z(psi, dh, Delta, Ht):
    

#Ht : hauteur sur laquelle on veut intégrer 
#notre valeur (peut n'être qu'une couche)

    si = psi.shape
    nl0 = si[0]
    

    dhl = np.copy(dh)
    if nl0 == 2:
        dhl = np.array([(dh[0]+dh[1])/2,(dh[0]+dh[1])/2])

    if psi.ndim == 3 :
        psiz = np.sum(psi*dhl[:,None,None], axis=0)/Ht
    
    elif psi.ndim == 2 :
        psiz = np.sum(psi*dhl[:,None], axis=0)/Ht
    
    
        

    return psiz
    

In [4]:
def get_psi_ke(p, f0):
    
    '''
    parameters
    
    p : array[nz, nx, ny]
    f0 : scalar 
    
    '''
    
    nd = p.ndim
    
    if nd < 3:
        print("not handling less than 3D arrays")
        sys.exit(1)
        
    
    psi = qg.interp_on_c(p)/f0
    
    s_z,s_x,s_y = psi.shape
    nz = 0
    psi_0 = np.zeros((s_z,s_x,s_y))
    
    for nz in range (0,s_z-1):
        psi_0[nz,:,:] = np.ones((s_x,s_y))*psi[nz,0,0]
    
    return psi

In [5]:
#moyenne sur le temps

'''
f : netcdf file
f0 : coriolis parameter
si_x, si_y, si_t : size of psi
'''

def mean_psi(f, f0, si_x, si_y, si_t):
    psi_me = 0.0*np.zeros((si_x,si_y))
    n_me = 0
    nt = 0
    for nt in range(0,si_t):
        psi = get_psi(f.variables['p'][nt,:,:,:].copy(),f0)
        psi_me = psi_me + psi
        n_me   = n_me + 1

    return psi_me/n_me
    

In [5]:
#moyenne sur le temps avec test generator class

'''
f : netcdf file
f0 : coriolis parameter
si_x, si_y, si_t : size of psi
'''

def mean_psi_test(f, f0, si_x, si_y, si_t):
    psi_me = 0.0*np.zeros((si_x,si_y))
    n_me = 0
    nt = 0
    gen = (get_psi(f.variables['p'][nt,:,:,:].copy(),f0) for nt in range(0,si_t))
    for psi in gen :
        psi_me = psi_me + psi
        n_me   = n_me + 1

    return psi_me/n_me
    

In [ ]:
# moyenne temporelle temps de KE psi turbulent

'''
f : netcdf file
f0 : coriolis parameter
si_x, si_y, si_t : size of psi
psi_me : moyenne temporelle de psi
'''

def mean_psi_turb(f1, f0, psi_me, si_x, si_y, si_t, Delta ):
    ke_turb_me  = 0.0*np.zeros((si_y,si_x))
    nt=0
    for nt in range(0,si_t):
        psi_tu = get_psi(f1.variables['p'][nt,:,:,:].copy(), f0) - psi_me
        ke_turb_me = ke_turb_me + qg.comp_ke(psi_tu, Delta) 
    
    return ke_turb_me/si_t